# Script Teste

_Script_ que define a classe `teste_cepespdata`

In [5]:
from teste_cepespdata import teste_cepespdata

In [17]:
inst = teste_cepespdata(2018, 3, agregacao_regional = 6, test = False)

Ano: 2018
Cargo: 3 
Banco Teste: False



In [18]:
inst.url_coligacao

'http://cepesp.io/api/consulta/legendas?format=csv&cargo=3&anos[]=2018'

Primeiro você tem que chamar esse call para baixar os dados e depois você faz o teste que quiser

In [19]:
inst.call_teste_geral()

Starting test...


In [27]:
inst.qtde_votos_uf

qtde_votos  qtd_aptos  qtd_comparecimento
ano_eleicao num_turno uf                                           
2018        1         AC      444644     547358              443429
                      AL     1224666    2187228             1692875
                      AM     2554003    2427111             1957418
                      AP      277227     511824              426382
                      BA     9768774   10390247             8235310
                      CE     5531006    6342532             5243490
                      DF     1691392    2081218             1691392
                      ES     2365861    2753585             2223207
                      GO     5352911    4453034             3553706
                      MA     3616086    4536193             3604676
                      MG    12283852   15695210            12211254
                      MS     2464604    1876929             1478642
                      MT     1772092    2329374             1757368
                      PA     4400611    5497589             4400611
                      PB     2435477    2866493             2435477
                      PE     6929719    6567535             5391639
                      PI     3321253    2370010             1997587
                      PR     6623157    7968409             6616901
                      RJ     9964606   12401199             9474441
                      RN     2158487    2372548             1966450
                      RO      912914    1175133              912914
                      RR      420756     333153              286754
                      RS     6835866    8350718             6835866
                      SC     4702556    5068421             4241667
                      SE     1279582    1576352             1279582
                      TO      831214    1038754              831214
            2         AM     2941884    2427159             1907930
                      AP      260428     511826              394703
                      DF     1687230    2081505             1687230
                      MG    12064283   15695460            12064283
                      MS     2743106    1877020             1449374
                      PA     4217054    5497638             4217054
                      RJ     9416708   12401560             9416708
                      RN     1942196    2372579             1942196
                      RO      881834    1175173              881834
                      RR      389804     333166              270315
                      RS     6774869    8350956             6774869
                      SC     4213519    5068559             4213519
                      SE     1264065    1576420             1264065